### There are mianly three retrieval technique in RAG 
Here arer some mentioned below

1) Exact match word
2) Embedding match based
3) Hybrid 

### Exact match search - using simple approach

In [70]:
# -------------------------------
# Imports
# -------------------------------
from typing import List
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever
from pydantic import PrivateAttr

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# -------------------------------
# 1️⃣ Custom Exact Match Retriever
# -------------------------------
class ExactMatchRetriever(BaseRetriever):
    _documents: List[Document] = PrivateAttr()

    def __init__(self, documents: List[Document], **kwargs):
        super().__init__(**kwargs)
        self._documents = documents

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_lower = query.lower()
        return [
            doc for doc in self._documents
            if any(word in doc.page_content.lower() for word in query_lower.split())
        ]

# -------------------------------
# 2️⃣ Prepare Documents
# -------------------------------
docs = [
    Document(page_content="Python is a programming language."),
    Document(page_content="LlamaIndex is a framework for building LLM apps."),
    Document(page_content="FAISS provides similarity search."),
]

retriever = ExactMatchRetriever(docs)

# -------------------------------
# 3️⃣ Setup LLM + Prompt
# -------------------------------
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the document combination chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# -------------------------------
# 4️⃣ Create Retrieval Chain
# -------------------------------
chain = create_retrieval_chain(retriever, question_answer_chain)

# -------------------------------
# 5️⃣ Query Example
# -------------------------------
query = "What is LlamaIndex?"
result = chain.invoke({"input": query})

print("Answer:\n", result)


C:\Users\aniln\AppData\Local\Temp\ipykernel_19960\698202827.py:17: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ExactMatchRetriever(BaseRetriever):
2025-10-13 14:01:36,283 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer:
 {'input': 'What is LlamaIndex?', 'context': [Document(metadata={}, page_content='Python is a programming language.'), Document(metadata={}, page_content='LlamaIndex is a framework for building LLM apps.'), Document(metadata={}, page_content='FAISS provides similarity search.')], 'answer': 'LlamaIndex is a framework for building LLM apps.'}


## Using BM25

In [74]:
from typing import List
from rank_bm25 import BM25Okapi
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import PrivateAttr

# -------------------------------
# 3️⃣ Prepare Documents
# -------------------------------
docs = [
    Document(page_content="Python is a programming language."),
    Document(page_content="LlamaIndex is a framework for building LLM apps."),
    Document(page_content="FAISS provides similarity search."),
]

# -------------------------------
# 4️⃣ BM25 Retriever with PrivateAttr
# -------------------------------
class BM25Retriever(BaseRetriever):
    _documents: List[Document] = PrivateAttr()
    _tokenized_docs: List[List[str]] = PrivateAttr()
    _bm25: BM25Okapi = PrivateAttr()

    def __init__(self, documents: List[Document], **kwargs):
        super().__init__(**kwargs)
        self._documents = documents
        self._tokenized_docs = [doc.page_content.lower().split() for doc in documents]
        self._bm25 = BM25Okapi(self._tokenized_docs)

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_tokens = query.lower().split()
        scores = self._bm25.get_scores(query_tokens)
        ranked_docs = [doc for _, doc in sorted(zip(scores, self._documents), reverse=True)]
        return ranked_docs[:3]  # top 3 documents

retriever = BM25Retriever(docs)

# -------------------------------
# 5️⃣ Setup LLM + Prompt
# -------------------------------
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

# -------------------------------
# 6️⃣ Create Retrieval Chain
# -------------------------------
chain = create_retrieval_chain(retriever, question_answer_chain)

# -------------------------------
# 7️⃣ Query Example
# -------------------------------
query = "What is LlamaIndex?"
result = chain.invoke({"input": query})

print("Answer:\n", result)

C:\Users\aniln\AppData\Local\Temp\ipykernel_19960\2147908413.py:23: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class BM25Retriever(BaseRetriever):
2025-10-13 14:03:48,128 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer:
 {'input': 'What is LlamaIndex?', 'context': [Document(metadata={}, page_content='Python is a programming language.'), Document(metadata={}, page_content='LlamaIndex is a framework for building LLM apps.'), Document(metadata={}, page_content='FAISS provides similarity search.')], 'answer': 'LlamaIndex is a framework for building LLM apps.'}


### Ebmedding based 

In [76]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from getpass import getpass

# -------------------------------
# 3️⃣ Set OpenAI API Key
# -------------------------------
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# -------------------------------
# 4️⃣ Prepare Documents
# -------------------------------
docs = [
    Document(page_content="Python is a programming language."),
    Document(page_content="LlamaIndex is a framework for building LLM apps."),
    Document(page_content="FAISS provides similarity search."),
]

# -------------------------------
# 5️⃣ Create embeddings and FAISS vector store
# -------------------------------
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# -------------------------------
# 6️⃣ Setup Retriever
# -------------------------------
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # top 3 similar documents
)

# -------------------------------
# 7️⃣ Setup LLM and Prompt
# -------------------------------
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

# -------------------------------
# 8️⃣ Create Retrieval Chain
# -------------------------------
chain = create_retrieval_chain(retriever, question_answer_chain)

# -------------------------------
# 9️⃣ Query Example
# -------------------------------
query = "What is LlamaIndex?"
response = chain.invoke({"input": query})

print("Answer:\n", response)

C:\Users\aniln\AppData\Local\Temp\ipykernel_19960\3102930292.py:28: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
2025-10-13 14:07:06,672 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-13 14:07:06,711 - INFO - Loading faiss with AVX512 support.
2025-10-13 14:07:06,713 - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-10-13 14:07:06,713 - INFO - Loading faiss with AVX2 support.
2025-10-13 14:07:06,826 - INFO - Successfully loaded faiss with AVX2 support.
2025-10-13 14:07:07,564 - INFO - HTTP Request: POST https://api.openai.co

Answer:
 {'input': 'What is LlamaIndex?', 'context': [Document(id='7ed30682-61b3-4a7b-aa84-ebd6f6904ec5', metadata={}, page_content='LlamaIndex is a framework for building LLM apps.'), Document(id='c9a3998b-2597-487e-8a00-34a875259cc3', metadata={}, page_content='FAISS provides similarity search.'), Document(id='932a6196-1746-4466-bf40-d9f9a581c2ea', metadata={}, page_content='Python is a programming language.')], 'answer': 'LlamaIndex is a framework for building LLM apps.'}


### Hybrid Based

In [80]:
import os
from getpass import getpass
from typing import List

from rank_bm25 import BM25Okapi
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever
from pydantic import PrivateAttr
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain

# -------------------------------
# 3️⃣ Set OpenAI API Key
# -------------------------------
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# -------------------------------
# 4️⃣ Prepare Documents
# -------------------------------
docs = [
    Document(page_content="Python is a programming language."),
    Document(page_content="LlamaIndex is a framework for building LLM apps."),
    Document(page_content="FAISS provides similarity search."),
]

# -------------------------------
# 5️⃣ BM25 Retriever
# -------------------------------
class BM25Retriever:
    def __init__(self, documents: List[Document], top_k: int = 3):
        self._documents = documents
        self.top_k = top_k
        self.tokenized_docs = [doc.page_content.lower().split() for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_docs)

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_tokens = query.lower().split()
        scores = self.bm25.get_scores(query_tokens)
        ranked_docs = [doc for _, doc in sorted(zip(scores, self._documents), reverse=True)]
        return ranked_docs[:self.top_k]

# -------------------------------
# 6️⃣ Embedding Retriever (FAISS)
# -------------------------------
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
embedding_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# -------------------------------
# 7️⃣ Hybrid Retriever
# -------------------------------
class HybridRetriever(BaseRetriever):
    _bm25_retriever: BM25Retriever = PrivateAttr()
    _embedding_retriever: BaseRetriever = PrivateAttr()

    def __init__(self, bm25_retriever: BM25Retriever, embedding_retriever: BaseRetriever, **kwargs):
        super().__init__(**kwargs)
        self._bm25_retriever = bm25_retriever
        self._embedding_retriever = embedding_retriever

    def get_relevant_documents(self, query: str) -> List[Document]:
        bm25_docs = self._bm25_retriever.get_relevant_documents(query)
        embedding_docs = self._embedding_retriever.get_relevant_documents(query)
        # Merge and remove duplicates
        seen = set()
        merged_docs = []
        for doc in bm25_docs + embedding_docs:
            if doc.page_content not in seen:
                merged_docs.append(doc)
                seen.add(doc.page_content)
        return merged_docs

# Instantiate hybrid retriever
bm25_retriever = BM25Retriever(docs, top_k=2)
retriever = HybridRetriever(bm25_retriever, embedding_retriever)

# -------------------------------
# 8️⃣ LLM and Prompt Setup
# -------------------------------
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

# -------------------------------
# 9️⃣ Create Retrieval Chain
# -------------------------------
chain = create_retrieval_chain(retriever, question_answer_chain)

# -------------------------------
# 🔟 Query Example
# -------------------------------
query = "What is LlamaIndex?"
response = chain.invoke({"input": query})

print("Answer:\n", response)

2025-10-13 14:10:38,425 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
C:\Users\aniln\AppData\Local\Temp\ipykernel_19960\804332345.py:57: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class HybridRetriever(BaseRetriever):
C:\Users\aniln\AppData\Local\Temp\ipykernel_19960\804332345.py:68: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  embedding_docs = self._embedding_retriever.get_relevant_documents(query)
2025-10-13 14:10:38,930 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-13 14:10:39,666 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer:
 {'input': 'What is LlamaIndex?', 'context': [Document(metadata={}, page_content='Python is a programming language.'), Document(metadata={}, page_content='LlamaIndex is a framework for building LLM apps.'), Document(id='3ff3af9a-5d4e-4d36-9f92-f2ba9f83e205', metadata={}, page_content='FAISS provides similarity search.')], 'answer': 'LlamaIndex is a framework for building LLM apps.'}
